In [1]:
import sys
sys.path.append('../')

import pandas as pd 
import numpy as np
from sklearn.neighbors import KernelDensity
from feature_engineering import generate_features
from sklearn.model_selection import train_test_split
import numpy as np
from utils.bball_utils import load_config, retrieve_data
import matplotlib.pyplot as plt
import datetime as dt
from generate_pmf import generate_pmf, get_weighted_counts, smooth_probabilities
# https://kdepy.readthedocs.io/en/latest/examples.html

In [2]:
query = "SELECT * FROM basketball.stat_sheet ss"
data = retrieve_data(query)
input_data = pd.Series({
    "player": "Zion Williamson",
    "team": "NOP",
    "opponent": "MIA",
    "home": "NOP",
    "date": dt.date(2023, 6, 1),
    "points": 15,
    "total_rebounds": 8,
    "assists": 3
})
features = ["same_opponent", "same_player", "same_venue_status", "days_since_game"]
weights = [-1] * len(features)

In [3]:
temp = generate_pmf(data, weights, features, input_data)

In [4]:
np.shape(temp)

(72, 31, 25)

In [5]:
# what actual dimensions do i want?
# n_i,n_j,n_k,n_i,n_j,n_k  splits the tuples into double basically... because

In [6]:
n_i, n_j, n_k = temp.shape

# Generate grid coordinates
grid_coords = np.indices((n_i, n_j, n_k))

In [7]:
np.shape(temp)

(72, 31, 25)

In [8]:
grid_coords = np.transpose(grid_coords, axes=(1, 2, 3, 0))  # Shape: (n_i, n_j, n_k, 3)

# Flatten grid_coords for broadcasting
grid_coords_flat = grid_coords.reshape(n_i, n_j, n_k, -1)

# Calculate distances using broadcasting
distances = np.sqrt(np.sum((grid_coords_flat[:, :, :, np.newaxis, :] - grid_coords_flat[np.newaxis, :, :, :, :]) ** 2, axis=-1))

ValueError: operands could not be broadcast together with shapes (72,31,25,1,3) (1,72,31,25,3) 

In [11]:
grid_coords = np.indices((n_i, n_j, n_k))
grid_coords = np.transpose(grid_coords, axes=(1, 2, 3, 0))  # Shape: (n_i, n_j, n_k, 3)

# Flatten grid_coords for broadcasting
grid_coords_flat = grid_coords.reshape(-1, 3)  # Shape: (n_i * n_j * n_k, 3)

# Calculate distances using broadcasting
distances = np.sqrt(np.sum((grid_coords_flat[:, np.newaxis] - grid_coords_flat[np.newaxis]) ** 2, axis=-1))

# Reshape distances array to (n_i, n_j, n_k, n_i, n_j, n_k)
distances = distances.reshape(n_i, n_j, n_k, n_i, n_j, n_k)

print(distances.shape) 

MemoryError: Unable to allocate 34.8 GiB for an array with shape (55800, 55800, 3) and data type int32

In [13]:
from sklearn.neighbors import KernelDensity

In [14]:
KernelDensity(kernel='gaussian', bandwidth=0.5).fit(temp)

ValueError: Found array with dim 3. KernelDensity expected <= 2.

In [19]:
from scipy import stats

In [21]:
stats.gaussian_kde(temp.T)

ValueError: too many values to unpack (expected 2)

In [22]:
np.shape(temp)

(72, 31, 25)

In [26]:
import statsmodels.api as sm

In [28]:
temp

array([[[ 2.17997348e-04,  9.03459374e-05,  9.01375269e-05, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [ 2.18309964e-04,  2.17997348e-04,  2.17997348e-04, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [ 2.17997348e-04,  2.17893143e-04,  9.05543479e-05, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        ...,
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00]],

       [[ 2.17997348e-04,  2.06951594e-04,  8.23221344e-05, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [ 2.18309964e-04,  2.14766986e-04,  2.17788938e-04, ...,
         -0.00000000e+00, -0.00000000e

In [27]:
sm.nonparametric.KDEMultivariate(data=temp, var_type='cc', bw='normal_reference')

ValueError: cannot reshape array of size 55800 into shape (72,2)

>>> import statsmodels.api as sm
>>> nobs = 300
>>> np.random.seed(1234)  # Seed random generator
>>> c1 = np.random.normal(size=(nobs,1))
>>> c2 = np.random.normal(2, 1, size=(nobs,1))


In [29]:
import statsmodels.api as sm
nobs = 300
np.random.seed(1234)  # Seed random generator
c1 = np.random.normal(size=(nobs,1))
c2 = np.random.normal(2, 1, size=(nobs,1))


In [33]:
dens_u = sm.nonparametric.KDEMultivariate(data=[c1,c1,c2],
    var_type='cc', bw='normal_reference')

ValueError: cannot reshape array of size 900 into shape (3,2)

In [32]:
dens_u

KDE instance
Number of variables: k_vars = 2
Number of samples:   nobs = 300
Variable types:      cc
BW selection method: normal_reference

In [ ]:
# https://stackoverflow.com/questions/66219947/3d-data-contour-ploting-using-a-kde
stats.gaussian_kde(np.array([X, Y]), weights=Z)

In [34]:
stats.gaussian_kde(temp)

ValueError: too many values to unpack (expected 2)